<a href="https://colab.research.google.com/github/manasarthak/Emotion-classification-using-physiological-signal/blob/main/Feature_Extraction.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.utils import io
import numpy as np
import collections

from sklearn.preprocessing import StandardScaler,MinMaxScaler
from sklearn.utils import shuffle

import scipy.io
from scipy import signal,integrate
from scipy import stats
from scipy.stats import skew,kurtosis,entropy,iqr
import matplotlib.pyplot as plt
import math
import antropy as ap

In [2]:
def load_data(dim):
    if dim=='valence':
        labels_all=np.load('/content/drive/MyDrive/DEAP/valence/' + 'all_valence_labels.npy',allow_pickle=True)
        data_all=np.load('/content/drive/MyDrive/DEAP/valence/' + 'all_valence_data.npy',allow_pickle=True)
        print("Valence :",labels_all.shape,data_all.shape)
    elif dim=='arousal':
        labels_all=np.load('/content/drive/MyDrive/DEAP/arousal/'+'all_arousal_labels.npy',allow_pickle=True)
        data_all=np.load('/content/drive/MyDrive/DEAP/arousal/'+'all_arousal_data.npy',allow_pickle=True)
        print("Arousal: ",labels_all.shape,data_all.shape)
    return labels_all,data_all

In [3]:
labels_all_val,data_all_val=load_data(dim='valence')

Valence : (1280,) (1280, 32, 6400)


In [4]:
labels_all_ar,data_all_ar=load_data(dim='arousal')

Arousal:  (1280,) (1280, 32, 6400)


Statistical/Time Domain Features(Linear and Complex)

In [5]:
def mean(x):
  return (np.mean(x,axis=-1));

def std(data):
    return np.std(data,axis=-1)#standard deviation

def ptp(data):
    return np.ptp(data,axis=-1)#range

def minim(data):
      return np.min(data,axis=-1)

def maxim(data):
      return np.max(data,axis=-1)

def argminim(data):
      return np.argmin(data,axis=-1)#indices of the minimum value(related to time)

def argmaxim(data):
      return np.argmax(data,axis=-1)

def mean_square(data):
      return np.mean(data**2,axis=-1)

def rms(data): 
      return  np.sqrt(np.mean(data**2,axis=-1)) #root mean square 

def abs_diffs_signal(data):
    return np.sum(np.abs(np.diff(data,axis=-1)),axis=-1)

def skewness(data):
    return skew(data,axis=-1)#measure of assysmetry of a distribution

def kkurtosis(data):
    return stats.kurtosis(data,axis=-1)#measure of tailedness of a distribution

def calc_entropy(data):
  return np.apply_along_axis(entropy,-1,data);

def calc_IQR(data):
  return np.apply_along_axis(iqr,-1,data);

def hjorth_activity(data):
  return np.var(data,axis=-1);

def hjorth_mobility(data):
  return np.divide(np.std(np.diff(data, axis=-1)),np.std(data, axis=-1));

def hjorth_complexity(data):
  return np.divide(hjorth_mobility(np.diff(data, axis=-1)), hjorth_mobility(data));

def petrosian_fractD(data):#Original code from the `pyrem <https://github.com/gilestrolab/pyrem>`_package by Quentin Geissmann.
  N=data.shape[-1];
  nzc_derivative=ap.num_zerocross(np.diff(data,axis=-1),axis=-1);#counting the number of zero cross in the first derivative of signal
  pfd = np.log10(N) / (np.log10(N) + np.log10(N / (N + 0.4 * nzc_derivative)));
  return pfd;

def katz_fd(data):#Original code from the `mne-features <https://mne.tools/mne-features/>`_package by Jean-Baptiste Schiratti and Alexandre Gramfort.
  dists = np.abs(np.diff(data, axis=-1))
  ll = dists.sum(axis=-1)
  ln = np.log10(ll / dists.mean(axis=-1))
  aux_d = data - np.take(data, indices=[0], axis=-1)
  d = np.max(np.abs(aux_d), axis=-1)
  kfd = np.squeeze(ln / (ln + np.log10(d / ll)))
  if not kfd.ndim:
      kfd = kfd.item()
  return kfd;

Frequency Domain Features

In [6]:
def spectral_entropy(x, sf, method='welch', nperseg=None, normalize=False, axis=-1):
  return ap.spectral_entropy(x,sf,method,nperseg,normalize,axis);

In [7]:
def concatenate_features(data):
    return np.concatenate((mean(data),std(data),ptp(data),minim(data),maxim(data),argminim(data),argmaxim(data),
                          mean_square(data),rms(data),abs_diffs_signal(data),
                          skewness(data),kkurtosis(data),calc_entropy(data),calc_IQR(data),hjorth_activity(data),hjorth_mobility(data)
                          ,hjorth_complexity(data),petrosian_fractD(data),katz_fd(data),spectral_entropy(data,128)),axis=-1);

In [9]:
from tqdm.notebook import tqdm
features_val=[]
for data in tqdm(data_all_val):
    features_val.append(concatenate_features(data))
features_val=np.array(features_val)

  0%|          | 0/1280 [00:00<?, ?it/s]

In [10]:
features_val.shape

(1280, 640)

In [11]:
features_ar=[]
for data in tqdm(data_all_ar):
    features_ar.append(concatenate_features(data))
features_ar=np.array(features_ar)

  0%|          | 0/1280 [00:00<?, ?it/s]

In [12]:
features_ar.shape

(1280, 640)

In [13]:
np.save('/content/drive/MyDrive/DEAP/valence/' + 'features_val.npy', features_val)
np.save('/content/drive/MyDrive/DEAP/arousal/' + 'features_ar.npy', features_ar)